In [ ]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.python import BranchPythonOperator 
from airflow.operators.dummy import DummyOperator
from datetime import datetime, timedelta
import pandas as pd 
import numpy as np 
import os
import requests 
import time 
import json 


def get_data(**kwargs):
    tickers = kwargs["tickers"]
    api_key = kwargs["api_key"]
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={tickers}&apikey={api_key}'
    r = requests.get(url)
    try:
        data = r.json()
        path = "C:/Users/salvo/OneDrive/Desktop/airflow/DATA_CENTER/DATA_LAKE"
        with open(path + "stock_market_rawdata" +tickers+'_'+ str(time.time()), "w") as f:
            json.dump(data, f) 
    except:
        print("Error")

#crontab notation  https://crontab.guru/#30_11_18_10_3

default_dag_args = {
    "start_date": datetime(2023,9,1),
    "email_on_failure": False,
    "email_on_retry": False,
    "retryes": 1,
    "retray_delay": timedelta(minutes=5),
    "project_id": 1
}

with DAG("market_data_alphavantage_dag", schedule_interval = '@daily', catchup=False, default_args = default_dag_args) as dag_python:

    task_0 = PythonOperator(task_id = "get_market_data", python_callable = get_data, op_kwargs = {'tickers' : "IBM"})